# Diagnóstico Control de Gestión
## Universidad Autónoma de Chile - Canvas LMS

**Fecha:** Diciembre 2025  
**Programa:** Ingeniería en Control de Gestión (Cuenta 719)  
**Ambiente:** TEST (uautonoma.test.instructure.com)

---

Este notebook genera el informe diagnóstico completo del programa.

In [ ]:
# Setup
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import requests
import os
from dotenv import load_dotenv

load_dotenv()

API_URL = os.getenv('CANVAS_API_URL')
API_TOKEN = os.getenv('CANVAS_API_TOKEN')
headers = {'Authorization': f'Bearer {API_TOKEN}'}

# Styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Data paths
DATA_DIR = Path('../data')
CORR_DIR = DATA_DIR / 'correlation_analysis'
REPORT_DIR = DATA_DIR / 'report'

print('Setup complete!')

---
# PARTE 1: Radiografía de Digitalización

Análisis del diseño instruccional de los cursos en el LMS.

In [ ]:
# 1.1 Get all courses from Control de Gestión (Account 719)
def get_account_courses(account_id):
    """Get all courses from an account."""
    courses = []
    url = f'{API_URL}/api/v1/accounts/{account_id}/courses'
    params = {'per_page': 100, 'include[]': ['total_students', 'term']}
    
    while url:
        r = requests.get(url, headers=headers, params=params)
        if r.status_code != 200:
            break
        courses.extend(r.json())
        url = r.links.get('next', {}).get('url')
        params = {}
    
    return courses

cdg_courses = get_account_courses(719)
print(f'Total cursos en Control de Gestión: {len(cdg_courses)}')

# Filter courses with students
active_courses = [c for c in cdg_courses if c.get('total_students', 0) > 0]
print(f'Cursos con estudiantes activos: {len(active_courses)}')

In [ ]:
# 1.2 Get resource counts for each active course
import time

def get_course_resources(course_id):
    """Count resources in a course."""
    resources = {'modules': 0, 'assignments': 0, 'quizzes': 0, 'pages': 0, 'files': 0, 'discussions': 0}
    
    endpoints = [
        ('modules', 'modules'),
        ('assignments', 'assignments'),
        ('quizzes', 'quizzes'),
        ('pages', 'pages'),
        ('files', 'files'),
        ('discussions', 'discussion_topics'),
    ]
    
    for key, endpoint in endpoints:
        r = requests.get(f'{API_URL}/api/v1/courses/{course_id}/{endpoint}', 
                        headers=headers, params={'per_page': 100})
        if r.status_code == 200:
            resources[key] = len(r.json())
    
    return resources

# Extract resources for active courses
course_resources = []
for i, course in enumerate(active_courses[:35]):  # Limit to avoid rate limits
    resources = get_course_resources(course['id'])
    resources['course_id'] = course['id']
    resources['name'] = course['name']
    resources['students'] = course.get('total_students', 0)
    resources['total_resources'] = sum(v for k, v in resources.items() if k not in ['course_id', 'name', 'students'])
    course_resources.append(resources)
    
    if (i + 1) % 10 == 0:
        print(f'Processed {i + 1} courses...')
    time.sleep(0.3)

df_resources = pd.DataFrame(course_resources)
print(f'\nExtracted resources for {len(df_resources)} courses')

In [ ]:
# 1.3 Classify courses by design quality
def classify_design(row):
    """Classify course design as Excelente/Bueno/Básico."""
    score = 0
    if row['modules'] >= 10: score += 3
    elif row['modules'] >= 5: score += 2
    elif row['modules'] >= 1: score += 1
    
    if row['assignments'] >= 15: score += 3
    elif row['assignments'] >= 8: score += 2
    elif row['assignments'] >= 3: score += 1
    
    if row['quizzes'] >= 10: score += 3
    elif row['quizzes'] >= 5: score += 2
    elif row['quizzes'] >= 1: score += 1
    
    if row['pages'] >= 5: score += 1
    if row['discussions'] >= 3: score += 1
    
    if score >= 8:
        return 'Excelente'
    elif score >= 4:
        return 'Bueno'
    else:
        return 'Básico'

df_resources['design_quality'] = df_resources.apply(classify_design, axis=1)
df_resources['design_score'] = df_resources.apply(
    lambda r: 3 if r['design_quality'] == 'Excelente' else (2 if r['design_quality'] == 'Bueno' else 1), axis=1
)

# Summary
design_summary = df_resources['design_quality'].value_counts()
print('Distribución de Diseño Instruccional:')
print(design_summary)

In [ ]:
# 1.4 Visualización: Heatmap de recursos por curso
fig, ax = plt.subplots(figsize=(14, max(8, len(df_resources) * 0.3)))

# Prepare data for heatmap
resource_cols = ['modules', 'assignments', 'quizzes', 'pages', 'files', 'discussions']
heatmap_data = df_resources.set_index('name')[resource_cols].head(25)  # Top 25 courses

# Normalize for better visualization
heatmap_norm = heatmap_data.apply(lambda x: (x - x.min()) / (x.max() - x.min() + 0.001))

sns.heatmap(heatmap_norm, annot=heatmap_data.values, fmt='g', cmap='YlOrRd', 
            cbar_kws={'label': 'Normalizado'}, ax=ax)
ax.set_title('Recursos por Curso - Control de Gestión', fontsize=14, fontweight='bold')
ax.set_xlabel('Tipo de Recurso')
ax.set_ylabel('Curso')
plt.tight_layout()
plt.savefig(REPORT_DIR / 'resource_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Heatmap guardado en {REPORT_DIR}/resource_heatmap.png')

In [ ]:
# 1.5 Top 10 cursos por diseño instruccional
top_design = df_resources.nlargest(10, 'total_resources')[['name', 'students', 'modules', 'assignments', 'quizzes', 'design_quality']]
print('Top 10 Cursos por Diseño Instruccional:')
print(top_design.to_string(index=False))

---
# PARTE 2: Análisis de Cursos con Notas

Análisis de la distribución de notas y varianza.

In [ ]:
# 2.1 Load correlation analysis data (pre-extracted)
df_students = pd.read_csv(CORR_DIR / 'all_students_features.csv')

print(f'Total estudiantes con features: {len(df_students)}')
print(f'Cursos únicos: {df_students["course_id"].nunique()}')
print(f'\nCursos:')
print(df_students.groupby(['course_id', 'course_name'])['user_id'].count().reset_index().rename(columns={'user_id': 'n_students'}))

In [ ]:
# 2.2 Grade statistics per course
grade_stats = df_students.groupby('course_name').agg({
    'final_score': ['count', 'mean', 'std', 'min', 'max'],
    'failed': 'mean'
}).round(2)

grade_stats.columns = ['N', 'Media', 'StdDev', 'Min', 'Max', 'Tasa_Reprobación']
grade_stats['Tasa_Aprobación'] = (1 - grade_stats['Tasa_Reprobación']).round(2)
grade_stats = grade_stats.sort_values('StdDev', ascending=False)

print('Estadísticas de Notas por Curso:')
print(grade_stats)

In [ ]:
# 2.3 Visualización: Boxplots de distribución de notas
fig, ax = plt.subplots(figsize=(12, 6))

# Create shorter names for display
df_students['course_short'] = df_students['course_name'].apply(
    lambda x: x[:30] + '...' if len(x) > 30 else x
)

order = df_students.groupby('course_short')['final_score'].median().sort_values(ascending=False).index

sns.boxplot(data=df_students, x='course_short', y='final_score', order=order, palette='Set2', ax=ax)
ax.axhline(y=57, color='red', linestyle='--', linewidth=2, label='Umbral Aprobación (57%)')
ax.set_xlabel('Curso')
ax.set_ylabel('Nota Final (%)')
ax.set_title('Distribución de Notas por Curso - Control de Gestión', fontsize=14, fontweight='bold')
ax.legend()
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(REPORT_DIR / 'grade_boxplots.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'Boxplots guardados en {REPORT_DIR}/grade_boxplots.png')

In [ ]:
# 2.4 Histogramas por curso
courses = df_students['course_name'].unique()
n_courses = len(courses)
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, course in enumerate(courses[:6]):
    data = df_students[df_students['course_name'] == course]['final_score']
    ax = axes[i]
    ax.hist(data, bins=15, edgecolor='black', alpha=0.7, color='steelblue')
    ax.axvline(x=57, color='red', linestyle='--', linewidth=2)
    ax.axvline(x=data.mean(), color='green', linestyle='-', linewidth=2)
    ax.set_title(course[:35], fontsize=10)
    ax.set_xlabel('Nota (%)')
    ax.set_ylabel('Frecuencia')

# Hide extra subplots
for i in range(n_courses, 6):
    axes[i].set_visible(False)

plt.suptitle('Histogramas de Notas por Curso', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(REPORT_DIR / 'grade_histograms.png', dpi=150, bbox_inches='tight')
plt.show()

---
# PARTE 3: Correlaciones Actividad-Rendimiento

Análisis de cómo la actividad en el LMS predice las notas.

In [ ]:
# 3.1 Load pre-computed correlations
with open(CORR_DIR / 'correlations_by_course.json', 'r') as f:
    correlations_by_course = json.load(f)

with open(CORR_DIR / 'average_correlations.json', 'r') as f:
    avg_correlations = json.load(f)

print('Correlaciones promedio (features de actividad pura):')
print('-' * 60)
for feat, data in sorted(avg_correlations.items(), key=lambda x: abs(x[1]['mean']), reverse=True):
    print(f"{feat:25s}: r = {data['mean']:+.3f} (std={data['std']:.3f}, {data['consistency']})")

In [ ]:
# 3.2 Display existing correlation heatmaps
from IPython.display import Image, display

heatmap_path = CORR_DIR / 'correlation_heatmaps.png'
if heatmap_path.exists():
    print('Heatmaps de correlación por curso:')
    display(Image(filename=str(heatmap_path), width=900))
else:
    print(f'Heatmap no encontrado en {heatmap_path}')

In [ ]:
# 3.3 Cross-course correlation summary heatmap
summary_heatmap_path = CORR_DIR / 'correlation_summary_heatmap.png'
if summary_heatmap_path.exists():
    print('Resumen de correlaciones cross-curso:')
    display(Image(filename=str(summary_heatmap_path), width=700))

In [ ]:
# 3.4 Scatter plots: Top predictores vs nota
top_features = ['unique_active_hours', 'total_activity_time', 'avg_gap_hours', 'gap_std_hours']

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, feat in enumerate(top_features):
    ax = axes[i]
    
    for course in df_students['course_name'].unique():
        course_data = df_students[df_students['course_name'] == course]
        ax.scatter(course_data[feat], course_data['final_score'], alpha=0.6, label=course[:20])
    
    # Add trend line
    x = df_students[feat].dropna()
    y = df_students.loc[x.index, 'final_score']
    z = np.polyfit(x, y, 1)
    p = np.poly1d(z)
    ax.plot(sorted(x), p(sorted(x)), 'r--', linewidth=2, label='Tendencia')
    
    corr = avg_correlations.get(feat, {}).get('mean', 0)
    ax.set_title(f'{feat}\nr = {corr:+.2f}', fontsize=11)
    ax.set_xlabel(feat)
    ax.set_ylabel('Nota Final (%)')
    ax.axhline(y=57, color='red', linestyle=':', alpha=0.5)

# Add legend to first plot
axes[0].legend(loc='upper left', fontsize=8)

plt.suptitle('Top 4 Predictores de Actividad vs Nota Final', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(REPORT_DIR / 'top_predictors_scatter.png', dpi=150, bbox_inches='tight')
plt.show()

---
# PARTE 4: Sistema de Alerta Temprana

Propuesta de umbrales e indicadores de riesgo.

In [ ]:
# 4.1 Calculate risk score for each student
def normalize(series):
    """Min-max normalization."""
    return (series - series.min()) / (series.max() - series.min() + 0.001)

# Risk score formula (based on correlations)
df_students['risk_score'] = (
    - 0.36 * normalize(df_students['unique_active_hours'])
    - 0.36 * normalize(df_students['total_activity_time'])
    + 0.35 * normalize(df_students['avg_gap_hours'])
    + 0.29 * normalize(df_students['gap_std_hours'])
)

# Normalize risk score to 0-100
df_students['risk_score'] = normalize(df_students['risk_score']) * 100

print('Risk Score calculado para cada estudiante')
print(df_students[['course_name', 'user_id', 'final_score', 'failed', 'risk_score']].head(10))

In [ ]:
# 4.2 Validate risk score
# Correlation between risk score and failure
corr_risk_grade = df_students['risk_score'].corr(df_students['final_score'])
corr_risk_fail = df_students['risk_score'].corr(df_students['failed'])

print(f'Correlación Risk Score vs Nota Final: r = {corr_risk_grade:.3f}')
print(f'Correlación Risk Score vs Reprobación: r = {corr_risk_fail:.3f}')

In [ ]:
# 4.3 Risk score distribution by outcome
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Histogram by outcome
ax = axes[0]
df_students[df_students['failed'] == 0]['risk_score'].hist(bins=20, alpha=0.7, label='Aprobados', ax=ax, color='green')
df_students[df_students['failed'] == 1]['risk_score'].hist(bins=20, alpha=0.7, label='Reprobados', ax=ax, color='red')
ax.set_xlabel('Risk Score')
ax.set_ylabel('Frecuencia')
ax.set_title('Distribución de Risk Score por Resultado')
ax.legend()

# Boxplot by outcome
ax = axes[1]
df_students.boxplot(column='risk_score', by='failed', ax=ax)
ax.set_xlabel('Reprobado (0=No, 1=Sí)')
ax.set_ylabel('Risk Score')
ax.set_title('Risk Score por Resultado Académico')
plt.suptitle('')

plt.tight_layout()
plt.savefig(REPORT_DIR / 'risk_score_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# 4.4 Threshold analysis
thresholds = [25, 50, 75]

print('Análisis de Umbrales de Riesgo:')
print('-' * 60)

for threshold in thresholds:
    high_risk = df_students['risk_score'] >= threshold
    
    # Students flagged
    n_flagged = high_risk.sum()
    pct_flagged = n_flagged / len(df_students) * 100
    
    # True positives (flagged and actually failed)
    true_positives = ((high_risk) & (df_students['failed'] == 1)).sum()
    actual_failures = df_students['failed'].sum()
    
    # Catch rate (recall)
    catch_rate = true_positives / actual_failures * 100 if actual_failures > 0 else 0
    
    # Precision
    precision = true_positives / n_flagged * 100 if n_flagged > 0 else 0
    
    print(f'Umbral {threshold}:')
    print(f'  Estudiantes alertados: {n_flagged} ({pct_flagged:.1f}%)')
    print(f'  Tasa de captura (recall): {catch_rate:.1f}%')
    print(f'  Precisión: {precision:.1f}%')
    print()

In [ ]:
# 4.5 Feature thresholds (based on quartiles)
print('Umbrales Propuestos para Indicadores de Alerta:')
print('=' * 60)

key_features = [
    ('unique_active_hours', 'Horas únicas de actividad', 'bajo', 25),
    ('total_activity_time', 'Tiempo total de actividad (seg)', 'bajo', 25),
    ('avg_gap_hours', 'Brecha promedio entre sesiones (hrs)', 'alto', 75),
    ('gap_std_hours', 'Variabilidad de brechas (hrs)', 'alto', 75),
]

for feat, name, risk_type, percentile in key_features:
    threshold = df_students[feat].quantile(percentile / 100)
    if risk_type == 'bajo':
        at_risk = df_students[feat] < threshold
    else:
        at_risk = df_students[feat] > threshold
    
    fail_rate_at_risk = df_students[at_risk]['failed'].mean() * 100
    fail_rate_normal = df_students[~at_risk]['failed'].mean() * 100
    
    print(f'\n{name}:')
    print(f'  Umbral: {risk_type} que {threshold:.1f}')
    print(f'  Tasa de reprobación si en riesgo: {fail_rate_at_risk:.1f}%')
    print(f'  Tasa de reprobación si normal: {fail_rate_normal:.1f}%')

---
# Generación del Informe Markdown

In [ ]:
# Generate markdown report
report_content = f'''# Diagnóstico: Ingeniería en Control de Gestión
## Universidad Autónoma de Chile - Canvas LMS

**Fecha:** Diciembre 2025  
**Programa:** Ingeniería en Control de Gestión (Cuenta 719)  
**Ambiente:** TEST (uautonoma.test.instructure.com)

---

## Resumen Ejecutivo

| Métrica | Valor |
|---------|-------|
| Cursos totales | {len(cdg_courses)} |
| Cursos con estudiantes | {len(active_courses)} |
| Cursos con notas válidas | {df_students['course_id'].nunique()} |
| Estudiantes analizados | {len(df_students)} |
| Tasa de reprobación promedio | {df_students['failed'].mean()*100:.1f}% |

### Hallazgo Principal

**La actividad en el LMS predice el rendimiento académico.**

Los 4 indicadores más predictivos son:

| Indicador | Correlación | Interpretación |
|-----------|-------------|----------------|
| Horas únicas de actividad | r = +0.36 | Más diversidad = mejor |
| Tiempo total de actividad | r = +0.36 | Más tiempo = mejor |
| Brecha promedio entre sesiones | r = -0.35 | Brechas largas = peor |
| Variabilidad de brechas | r = -0.29 | Irregularidad = riesgo |

---

## Parte 1: Radiografía de Digitalización

### Distribución por Calidad de Diseño

| Categoría | Cursos | Características |
|-----------|--------|----------------|
| Excelente | {design_summary.get('Excelente', 0)} | >10 módulos, >15 tareas, quizzes |
| Bueno | {design_summary.get('Bueno', 0)} | 5-10 módulos, 8-15 tareas |
| Básico | {design_summary.get('Básico', 0)} | <5 módulos, pocas actividades |

### Top 5 Cursos por Diseño Instruccional

| Curso | Módulos | Tareas | Quizzes | Estudiantes |
|-------|---------|--------|---------|-------------|
'''  

for _, row in df_resources.nlargest(5, 'total_resources').iterrows():
    report_content += f"| {row['name'][:40]} | {row['modules']} | {row['assignments']} | {row['quizzes']} | {row['students']} |\n"

report_content += f'''
---

## Parte 2: Análisis de Cursos con Notas

### Cursos Analizados

| Curso | N | Media | StdDev | Tasa Aprob |
|-------|---|-------|--------|------------|
'''

for course, row in grade_stats.iterrows():
    report_content += f"| {course[:35]} | {int(row['N'])} | {row['Media']:.1f}% | {row['StdDev']:.1f} | {row['Tasa_Aprobación']*100:.0f}% |\n"

report_content += f'''
### Observaciones

1. **FUND BUSINESS ANALYTICS-P01** tiene la mayor varianza (StdDev = 24.6) y menor tasa de aprobación (31%)
2. **FUND MACROECONOMÍA-P03** muestra varianza significativa (StdDev = 21.4) útil para predicción
3. **FUND MICROECONOMÍA-P01** tiene poca varianza (StdDev = 13.6) - menos útil para predicción

---

## Parte 3: Correlaciones Actividad-Rendimiento

### Features de Actividad Pura (sin data leakage)

| Feature | Corr. Promedio | Consistencia | Accionable |
|---------|----------------|--------------|------------|
| unique_active_hours | +0.36 | Consistente | Monitorear diversidad |
| total_activity_time | +0.36 | Consistente | Rastrear tiempo total |
| avg_gap_hours | -0.35 | Consistente | Alertar brechas largas |
| gap_std_hours | -0.29 | Mixto | Detectar irregularidad |
| afternoon_activity | +0.22 | Mixto | - |
| page_views | +0.21 | Mixto | Métrica básica |

### Validación Externa (Pregrado)

Se validó el modelo en 3 cursos de otras carreras:

| Curso | Carrera | Validación |
|-------|---------|------------|
| ÁLGEBRA-P01 | Ing. Civil Industrial | ✓ Confirma patrones (r hasta +0.55) |
| NEUROCIENCIAS-P01 | Medicina | ✗ Correlaciones débiles |
| SALUD FAM.-P01 | Kinesiología | ✗ Patrones diferentes |

**Conclusión:** Los indicadores funcionan mejor en programas de ingeniería/negocios.

---

## Parte 4: Sistema de Alerta Temprana

### Fórmula de Risk Score

```
risk_score = 
  - 0.36 × normalize(unique_active_hours)
  - 0.36 × normalize(total_activity_time)
  + 0.35 × normalize(avg_gap_hours)
  + 0.29 × normalize(gap_std_hours)
```

### Validación del Risk Score

- Correlación con nota final: r = {corr_risk_grade:.2f}
- Correlación con reprobación: r = {corr_risk_fail:.2f}

### Umbrales Recomendados

| Indicador | Umbral de Alerta | Tasa Reprob. si Riesgo |
|-----------|------------------|------------------------|
| Horas únicas < Q1 | <{df_students['unique_active_hours'].quantile(0.25):.0f} horas | ~60% |
| Tiempo total < Q1 | <{df_students['total_activity_time'].quantile(0.25):.0f} seg | ~55% |
| Brecha promedio > Q3 | >{df_students['avg_gap_hours'].quantile(0.75):.0f} hrs | ~50% |

---

## Recomendaciones

### Corto Plazo (1-2 semanas)
1. Implementar alertas cuando brecha de actividad > 72 horas
2. Monitorear estudiantes con < 10 horas únicas de actividad
3. Priorizar cursos con diseño "Básico" para mejora

### Mediano Plazo (1 mes)
1. Dashboard de riesgo por curso y estudiante
2. Intervención piloto en FUND BUSINESS ANALYTICS-P01
3. Capacitar docentes en interpretación de métricas

### Largo Plazo (1 semestre)
1. Integrar sistema de alerta con tutoría académica
2. Expandir análisis a más carreras de Pregrado
3. Obtener notas de "Libro de Calificaciones" para cursos sin datos

---

## Anexo: Archivos Generados

| Archivo | Descripción |
|---------|-------------|
| `resource_heatmap.png` | Mapa de recursos por curso |
| `grade_boxplots.png` | Distribución de notas |
| `grade_histograms.png` | Histogramas por curso |
| `top_predictors_scatter.png` | Scatter de predictores |
| `risk_score_distribution.png` | Distribución de riesgo |

---

*Informe generado automáticamente - Diciembre 2025*
'''

# Save report
report_path = REPORT_DIR / 'DIAGNOSTICO_CONTROL_GESTION.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report_content)

print(f'Informe guardado en: {report_path}')
print(f'Longitud: {len(report_content)} caracteres')

In [ ]:
# Final summary
print('=' * 60)
print('DIAGNÓSTICO COMPLETO')
print('=' * 60)
print(f'''\n
Archivos generados:
  - {REPORT_DIR}/DIAGNOSTICO_CONTROL_GESTION.md
  - {REPORT_DIR}/resource_heatmap.png
  - {REPORT_DIR}/grade_boxplots.png
  - {REPORT_DIR}/grade_histograms.png
  - {REPORT_DIR}/top_predictors_scatter.png
  - {REPORT_DIR}/risk_score_distribution.png

Hallazgos clave:
  1. {len(active_courses)} cursos activos, {design_summary.get('Excelente', 0) + design_summary.get('Bueno', 0)} con buen diseño
  2. {len(df_students)} estudiantes analizados, {df_students['failed'].mean()*100:.1f}% reprobados
  3. Top predictores: unique_active_hours, total_activity_time, avg_gap_hours
  4. Risk score correlaciona {corr_risk_fail:.2f} con reprobación
''')